In [1]:
import numba
from numba import types, typed

import numpy as np

In [5]:
spec_connection = [('key', types.ListType(types.int32)),
                   ('mutate_rate', types.float32),
                   ('mean_weight', types.float32),
                   ('std_weight', types.float32)]
@numba.jitclass(spec_connection)
class ConnectionNB:
    def __init__(key):
        self.key = key
        self.mutate_rate = mutate_rate
        self.mean_weight = 0.0
        self.std_weight = 0.0

In [10]:
nodes_dict_type = (types.int32, NodeNB)
connections_dict_type = (types.ListType(types.int32), ConnectionNB)

# A container class with:
# * member 'd' holding a typed dictionary of int64 -> unicode string (kv_ty)
# * member 'l' holding a typed list of float64
spec_genome = [('key', types.int32),
               ('n_input', types.int16),
               ('n_output', types.int16),
               ('node_genes', types.DictType(*nodes_dict_type)),
               ('connection_genes', types.DictType(*connections_dict_type)),
               ('fitness', types.float64)]



@numba.jitclass(spec_genome)
class GenomeNB:
    def __init__(self, key, n_input, n_output):
        self.key = key
        self.n_input = n_input
        self.n_output = n_output
        
        self.node_genes = typed.Dict.empty(*nodes_dict_type)
        self.connection_genes = typed.Dict.empty(*connections_dict_type)
        
        self.fitness = 0
    
    def get_key(self):
        return self.key
    
    def set_node_genes(self, node_genes):
        self.node_genes = node_genes
    
    def get_node_genes(self):
        return self.node_genes
    
    def set_connection_genes(self, connection_genes):
        self.connection_genes = connection_genes
    
    def get_connection_genes(self):
        return self.connection_genes
    
    def set_fitness(self, fitness):
        self.fitness = fitness
        
    def get_fitness(self):
        return self.fitness

In [2]:
n_nodes = 50
mutate_rate = 0.5

mean_bias_values = [-0.2, 0.0, 0.2]
std_bias_values = [0.05, 0.1, 0.5]

## Numba Based

In [3]:
spec_node = [('key', types.int32),
             ('mutate_rate', types.float32),
             ('mean_bias_values', types.float64[:]),
             ('std_bias_values', types.float64[:]),
             ('mean_bias', types.float64),
             ('std_bias', types.float64)]
#              ('mean_bias_values', types.ListType(types.float64)),
#              ('std_bias_values', types.ListType(types.float64)),
@numba.jitclass(spec_node)
class NodeNB:
    
    def __init__(self, key, mutate_rate, mean_bias_values, std_bias_values):
        self.key = key
        self.mutate_rate = mutate_rate
        self.mean_bias_values = mean_bias_values
        self.std_bias_values = std_bias_values
        self.mean_bias = 0.0
        self.std_bias = 0.0
        
        self._mutate_mean()
        self._mutate_std()
        
    def get_mean_bias(self):
        return self.mean_bias
    
    def get_std_bias(self):
        return self.std_bias
    
    def get_key(self):
        return self.key
    
    def mutate_mean(self):
        r = np.random.random()
        if r < self.mutate_rate:
            self._mutate_mean()
    
    def _mutate_mean(self):
        self.mean_bias = np.random.choice(self.mean_bias_values, size=1)[0]
        
    def mutate_std(self):
        r = np.random.random()
        if r < self.mutate_rate:
            self._mutate_std()
            
    def _mutate_std(self):
        self.std_bias = np.random.choice(self.std_bias_values, size=1)[0]

In [119]:
@numba.njit()
def get_numba_array(list_):
    return np.array(list_)

typed_mean_bias_values = get_numba_array(mean_bias_values)
typed_std_bias_values = get_numba_array(std_bias_values)

@numba.njit(debug=True)
def initialize_nodes(n_nodes, mutate_rate, mean_bias_values, std_bias_values):
    nodes = {}
    for i in range(n_nodes):
        node = NodeNB(i, 
                      mutate_rate, 
                      mean_bias_values, 
                      std_bias_values)

        nodes[i] = node
    return nodes

/home/alberto/.local/share/virtualenvs/Neuro-Evolution-BNN-_WcBvtZA/lib/python3.6/site-packages/numba/ir_utils.py:1969: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'list_' of function 'get_numba_array'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-119-3330e6ab0a04>", line 2:
@numba.njit()
def get_numba_array(list_):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


## Simple Python

In [108]:
class Node:
    
    def __init__(self, key, mutate_rate, mean_bias_values, std_bias_values):
        self.key = key
        self.mutate_rate = mutate_rate
        self.mean_bias_values = mean_bias_values
        self.std_bias_values = std_bias_values
        self.mean_bias = 0.0
        self.std_bias = 0.0
        
    def get_mean_bias(self):
        return self.mean_bias
    
    def get_std_bias(self):
        return self.std_bias
    
#     def get_key(self):
#         return self.key
    
    def mutate_mean(self):
        r = np.random.random()
        if r < self.mutate_rate:
            self.mean_bias = np.random.choice(self.mean_bias_values, size=1)[0]
        
    def mutate_std(self):
        r = np.random.random()
        if r < self.mutate_rate:
            self.std_bias = np.random.choice(self.std_bias_values, size=1)[0]

In [109]:
def initialize_nodes_simple(n_nodes, mutate_rate, mean_bias_values, std_bias_values):
    nodes = {}
    for i in range(n_nodes):
        node = Node(0, 
                  mutate_rate, 
                  mean_bias_values, 
                  std_bias_values)
        node.mutate_mean()
        node.mutate_std()
        nodes[i] = node
    return nodes

## Comparison

In [129]:
n_nodes = 5000000

In [130]:
%%time
nodes = initialize_nodes(n_nodes, mutate_rate, typed_mean_bias_values, typed_std_bias_values)

CPU times: user 4.96 s, sys: 357 ms, total: 5.32 s
Wall time: 5.35 s


In [131]:
# for i in range(100):
#     print(nodes[i].get_key())
#     print(nodes[i].get_mean_bias())

In [132]:
%%time
nodes_py = initialize_nodes_simple(n_nodes, mutate_rate, typed_mean_bias_values, typed_std_bias_values)

CPU times: user 1min 38s, sys: 909 ms, total: 1min 39s
Wall time: 1min 43s
